# Chapter 3: State Estimation

## Principles of Indoor Positioning and Indoor Navigation

---

### 📚 Learning Objectives

By the end of this notebook, you will be able to:

1. **Understand** the fundamental concepts of state estimation for positioning
2. **Implement** Least Squares methods (LS, WLS, Iterative LS, Robust LS)
3. **Apply** Kalman Filter (KF) for linear dynamic systems
4. **Use** Extended Kalman Filter (EKF) for nonlinear systems
5. **Compare** different estimators (EKF, UKF, Particle Filter)

### 📖 Book Reference

This notebook covers **Chapter 3: State Estimation** with implementations of:
- **Eq. (3.1)-(3.4)**: Least Squares Methods
- **Eq. (3.11)-(3.19)**: Kalman Filter
- **Eq. (3.21)-(3.22)**: Extended Kalman Filter
- **Eq. (3.24)-(3.30)**: Unscented Kalman Filter
- **Eq. (3.32)-(3.34)**: Particle Filter

---


## 🚀 Setup (Google Colab)

Run this cell first to set up the environment. This will:
1. Clone the IPIN Book Examples repository
2. Install the package
3. Import required modules

**Note:** First run takes ~30-60 seconds. Subsequent runs are instant.


In [ ]:
# ========================================
# IPIN Book Examples - Chapter 3: State Estimation
# Run in Google Colab or Local Jupyter
# ========================================

import os
import sys

# Check if running in Colab
IN_COLAB = 'google.colab' in sys.modules

if IN_COLAB:
    # Clone repository (only needed once per session)
    if not os.path.exists('IPIN_Book_Examples'):
        print("📥 Cloning IPIN Book Examples repository...")
        !git clone https://github.com/YOUR_USERNAME/IPIN_Book_Examples.git
        %cd IPIN_Book_Examples
        print("📦 Installing package...")
        !pip install -e . -q
    else:
        %cd IPIN_Book_Examples
        print("✅ Repository already cloned.")
else:
    # Running locally - ensure we're in the right directory
    if os.path.basename(os.getcwd()) == 'notebooks':
        os.chdir('..')
    print(f"📂 Working directory: {os.getcwd()}")

# Import core libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse, Circle

# Configure matplotlib for inline display
%matplotlib inline
plt.rcParams['figure.figsize'] = [12, 8]
plt.rcParams['figure.dpi'] = 100
plt.rcParams['font.size'] = 12

# Import IPIN core modules
from core.estimators import (
    linear_least_squares,
    weighted_least_squares,
    iterative_least_squares,
    robust_least_squares,
    KalmanFilter,
    ExtendedKalmanFilter,
    UnscentedKalmanFilter,
    ParticleFilter,
)

print("\n" + "="*60)
print("✅ Setup complete! All core modules loaded.")
print("="*60)


---

# Part 1: Least Squares Methods

## 1.1 Theory Overview

Least Squares (LS) estimation minimizes the sum of squared residuals between measurements and predictions.

### Linear Least Squares - Eq. (3.1)

Given the linear measurement model:
$$\mathbf{z} = \mathbf{H}\mathbf{x} + \mathbf{v}$$

The LS estimate is:
$$\hat{\mathbf{x}} = (\mathbf{H}^T \mathbf{H})^{-1} \mathbf{H}^T \mathbf{z}$$

### Weighted Least Squares - Eq. (3.2)

When measurements have different accuracies, we weight them:
$$\hat{\mathbf{x}} = (\mathbf{H}^T \mathbf{W} \mathbf{H})^{-1} \mathbf{H}^T \mathbf{W} \mathbf{z}$$

where $\mathbf{W} = \mathbf{R}^{-1}$ is the inverse of measurement covariance.

### Iterative Least Squares (Gauss-Newton) - Eq. (3.3)

For nonlinear problems, we linearize and iterate:
$$\mathbf{x}^{(k+1)} = \mathbf{x}^{(k)} + (\mathbf{H}^T \mathbf{H})^{-1} \mathbf{H}^T (\mathbf{z} - h(\mathbf{x}^{(k)}))$$

### Robust Least Squares - Eq. (3.4)

Uses robust loss functions (Huber, Cauchy, Tukey) to downweight outliers.


## 1.2 Positioning Scenario Setup

We'll use a common indoor positioning scenario: **2D positioning from range measurements** (Time-of-Arrival).

- 4 anchors at corners of a 10m × 10m room
- Unknown target position to estimate
- Range measurements from target to each anchor


In [ ]:
# Define positioning scenario
def setup_positioning_scenario():
    """Create a 2D positioning scenario with 4 anchors."""
    # Anchor positions at corners of 10m × 10m room
    anchors = np.array([
        [0.0, 0.0],   # Bottom-left
        [10.0, 0.0],  # Bottom-right
        [0.0, 10.0],  # Top-left
        [10.0, 10.0]  # Top-right
    ])
    
    # True target position (unknown to estimator)
    true_position = np.array([3.0, 4.0])
    
    return anchors, true_position

def compute_ranges(position, anchors, noise_std=0.0):
    """Compute ranges from position to anchors with optional noise."""
    true_ranges = np.linalg.norm(anchors - position, axis=1)
    if noise_std > 0:
        true_ranges += noise_std * np.random.randn(len(anchors))
    return true_ranges

# Set up scenario
anchors, true_position = setup_positioning_scenario()

print("📍 Positioning Scenario")
print("="*40)
print(f"Room size: 10m × 10m")
print(f"Number of anchors: {len(anchors)}")
print(f"\nAnchor positions:")
for i, a in enumerate(anchors):
    print(f"  Anchor {i}: ({a[0]:.1f}, {a[1]:.1f}) m")
print(f"\nTrue target position: ({true_position[0]:.1f}, {true_position[1]:.1f}) m")


In [ ]:
# Visualize the scenario
fig, ax = plt.subplots(figsize=(10, 10))

# Draw room boundary
ax.plot([0, 10, 10, 0, 0], [0, 0, 10, 10, 0], 'k-', linewidth=2, label='Room')

# Plot anchors
ax.scatter(anchors[:, 0], anchors[:, 1], s=300, c='blue', marker='^', 
           label='Anchors', zorder=5, edgecolors='black', linewidths=2)
for i, a in enumerate(anchors):
    ax.annotate(f'A{i}', (a[0]+0.3, a[1]+0.3), fontsize=12, fontweight='bold')

# Plot true position
ax.scatter(true_position[0], true_position[1], s=400, c='green', marker='*', 
           label='True Position', zorder=5, edgecolors='black', linewidths=2)

# Draw range circles
ranges = compute_ranges(true_position, anchors)
for i, (anchor, r) in enumerate(zip(anchors, ranges)):
    circle = Circle(anchor, r, fill=False, edgecolor='blue', alpha=0.3, linestyle='--')
    ax.add_patch(circle)

ax.set_xlabel('X (m)', fontsize=14)
ax.set_ylabel('Y (m)', fontsize=14)
ax.set_title('2D Positioning Scenario\n(Range circles from each anchor)', fontsize=16, fontweight='bold')
ax.legend(fontsize=12, loc='upper right')
ax.grid(True, alpha=0.3)
ax.set_xlim(-2, 12)
ax.set_ylim(-2, 12)
ax.set_aspect('equal')
plt.show()

print("\n📏 True ranges to each anchor:")
for i, r in enumerate(ranges):
    print(f"  Anchor {i}: {r:.3f} m")


## 1.3 Example: Linear Least Squares

The range measurement model is nonlinear:
$$r_i = \sqrt{(x - x_i)^2 + (y - y_i)^2}$$

We linearize around an initial guess $\mathbf{x}_0$:
$$\mathbf{H} = \frac{\partial \mathbf{r}}{\partial \mathbf{x}} = \begin{bmatrix} \frac{x_0 - x_1}{r_1} & \frac{y_0 - y_1}{r_1} \\ \vdots & \vdots \end{bmatrix}$$


In [ ]:
# Example 1: Linear Least Squares
print("="*70)
print("EXAMPLE 1: Linear Least Squares (LS) - Eq. (3.1)")
print("="*70)

# Generate noisy measurements
np.random.seed(42)
noise_std = 0.1  # 10 cm measurement noise
ranges = compute_ranges(true_position, anchors, noise_std=noise_std)

print(f"\nMeasurement noise: σ = {noise_std} m")
print(f"\nNoisy range measurements:")
for i, r in enumerate(ranges):
    true_r = np.linalg.norm(true_position - anchors[i])
    print(f"  Anchor {i}: {r:.3f} m (true: {true_r:.3f} m, error: {r-true_r:+.3f} m)")

# Initial guess (center of room)
x0 = np.array([5.0, 5.0])
print(f"\nInitial guess: ({x0[0]}, {x0[1]}) m")

# Linearize: compute Jacobian at x0
diff = x0 - anchors  # [4, 2]
ranges_at_x0 = np.linalg.norm(diff, axis=1, keepdims=True)  # [4, 1]
H = diff / ranges_at_x0  # Jacobian: ∂r/∂x

# Observation vector: residual = measured - predicted
b = ranges - np.linalg.norm(anchors - x0, axis=1)

# Solve Linear LS: x̂ = x0 + (H'H)⁻¹H'b
dx, P = linear_least_squares(H, b)
position_ls = x0 + dx

# Calculate error
error = np.linalg.norm(position_ls - true_position)

print(f"\n📊 Results:")
print(f"  True position:     ({true_position[0]:.3f}, {true_position[1]:.3f}) m")
print(f"  LS estimate:       ({position_ls[0]:.3f}, {position_ls[1]:.3f}) m")
print(f"  Position error:    {error:.4f} m")
print(f"\n  Covariance matrix:")
print(f"  {P}")
print(f"  Position std dev: ({np.sqrt(P[0,0]):.4f}, {np.sqrt(P[1,1]):.4f}) m")


## 1.4 Example: Robust Least Squares (Outlier Rejection)

In real environments, measurements can be corrupted by outliers (e.g., NLOS multipath). Robust estimators use loss functions that reduce the influence of outliers.


In [ ]:
# Example: Robust Least Squares
print("="*70)
print("EXAMPLE: Robust Least Squares (Outlier Rejection) - Eq. (3.4)")
print("="*70)

# Generate measurements with an OUTLIER
np.random.seed(42)
ranges_clean = compute_ranges(true_position, anchors, noise_std=0.1)
ranges_outlier = ranges_clean.copy()
ranges_outlier[2] += 3.0  # Add 3m outlier to anchor 2 (NLOS!)

print(f"\n⚠️ Added 3.0 m outlier to Anchor 2 (simulating NLOS)")
print(f"\nRange measurements:")
for i, (clean, outlier) in enumerate(zip(ranges_clean, ranges_outlier)):
    status = "❌ OUTLIER" if i == 2 else "✓"
    print(f"  Anchor {i}: {outlier:.3f} m (clean: {clean:.3f} m) {status}")

# Linearization
x0 = np.array([5.0, 5.0])
diff = x0 - anchors
ranges_at_x0 = np.linalg.norm(diff, axis=1, keepdims=True)
H = diff / ranges_at_x0
b = ranges_outlier - np.linalg.norm(anchors - x0, axis=1)

# Standard LS (corrupted by outlier)
dx_ls, _ = linear_least_squares(H, b)
position_ls_outlier = x0 + dx_ls

# Robust LS with different loss functions
methods = ['huber', 'cauchy', 'tukey']
results = {}

for method in methods:
    dx_robust, P_robust, weights = robust_least_squares(H, b, method=method, threshold=2.0)
    position_robust = x0 + dx_robust
    results[method] = {
        'position': position_robust,
        'error': np.linalg.norm(position_robust - true_position),
        'outlier_weight': weights[2]
    }

error_ls = np.linalg.norm(position_ls_outlier - true_position)

print(f"\n📊 Results:")
print(f"  True position:      ({true_position[0]:.3f}, {true_position[1]:.3f}) m")
print(f"  Standard LS:        ({position_ls_outlier[0]:.3f}, {position_ls_outlier[1]:.3f}) m  [error: {error_ls:.4f} m] ❌ Corrupted!")
print()
for method, result in results.items():
    pos = result['position']
    print(f"  {method.capitalize():8s} LS:      ({pos[0]:.3f}, {pos[1]:.3f}) m  [error: {result['error']:.4f} m]  outlier weight: {result['outlier_weight']:.3f}")

print(f"\n💡 Robust methods downweight the outlier (weight << 1.0) and recover the true position!")


---

# Part 2: Kalman Filter

## 2.1 Theory Overview

The Kalman Filter is the optimal estimator for **linear systems** with **Gaussian noise**.

**State transition:** x_k = F * x_{k-1} + w_k

**Measurement:** z_k = H * x_k + v_k

**Prediction Step (Eq. 3.11-3.12):** Propagate state and covariance

**Update Step (Eq. 3.17-3.19):** Correct with measurement using Kalman gain


## 2.2 Example: 1D Constant Velocity Tracking

We'll track a target moving in 1D with constant velocity, using noisy position measurements.

**State vector:** x = [position, velocity]


In [ ]:
# 1D Kalman Filter Example
print("="*70)
print("EXAMPLE: 1D Constant Velocity Tracking with Kalman Filter")
print("="*70)

# Simulation parameters
dt = 0.1  # Time step (seconds)
t_max = 10.0  # Total time (seconds)
n_steps = int(t_max / dt)

# True initial state: [position, velocity]
true_x0 = np.array([0.0, 2.0])  # Start at x=0, velocity=2 m/s

# State transition matrix (constant velocity model) - Eq. (3.11)
F = np.array([[1.0, dt], [0.0, 1.0]])

# Process noise covariance - Eq. (3.12)
q = 0.1  # Process noise intensity
Q = q * np.array([[dt**3/3, dt**2/2], [dt**2/2, dt]])

# Measurement matrix (observe position only) - Eq. (3.8)
H = np.array([[1.0, 0.0]])

# Measurement noise covariance
measurement_std = 0.5  # meters
R = np.array([[measurement_std**2]])

# Initial estimate (poor velocity estimate)
x0_est = np.array([0.0, 0.0])
P0 = np.diag([1.0, 5.0])  # High uncertainty in velocity

print(f"\nSimulation Parameters:")
print(f"  Time step: {dt} s, Duration: {t_max} s")
print(f"  True initial: pos={true_x0[0]} m, vel={true_x0[1]} m/s")
print(f"  Measurement noise: {measurement_std} m")


# Generate true trajectory and run Kalman Filter
np.random.seed(42)

# Generate true trajectory
true_states = [true_x0.copy()]
true_state = true_x0.copy()

for _ in range(n_steps):
    process_noise = np.random.multivariate_normal(np.zeros(2), Q)
    true_state = F @ true_state + process_noise
    true_states.append(true_state.copy())

# Generate noisy measurements
measurements = []
for state in true_states[1:]:
    true_measurement = H @ state
    noise = np.random.normal(0, measurement_std)
    measurements.append(true_measurement[0] + noise)

# Run Kalman Filter
print("\nRunning Kalman Filter...")
kf = KalmanFilter(F, Q, H, R, x0_est, P0)

estimates = [x0_est.copy()]
covariances = [P0.copy()]

for z in measurements:
    kf.predict(dt=dt)
    kf.update(np.array([z]))
    x_est, P_est = kf.get_state()
    estimates.append(x_est.copy())
    covariances.append(P_est.copy())

# Convert to arrays
true_states = np.array(true_states)
estimates = np.array(estimates)
time = np.arange(n_steps + 1) * dt

# Compute errors
position_errors = np.abs(estimates[:, 0] - true_states[:, 0])
velocity_errors = np.abs(estimates[:, 1] - true_states[:, 1])

print(f"\nResults:")
print(f"  Mean position error (after convergence): {np.mean(position_errors[20:]):.4f} m")
print(f"  Mean velocity error (after convergence): {np.mean(velocity_errors[20:]):.4f} m/s")


In [ ]:
# Visualization: Kalman Filter Results
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Plot 1: Position vs Time
ax = axes[0, 0]
ax.plot(time, true_states[:, 0], 'g-', linewidth=2, label='True Position')
ax.plot(time[1:], measurements, 'r.', alpha=0.5, markersize=4, label='Measurements')
ax.plot(time, estimates[:, 0], 'b-', linewidth=2, label='KF Estimate')

# Uncertainty bounds
pos_std = np.array([np.sqrt(P[0, 0]) for P in covariances])
ax.fill_between(time, estimates[:, 0] - 2*pos_std, estimates[:, 0] + 2*pos_std,
                alpha=0.2, color='blue', label='2-sigma bounds')

ax.set_xlabel('Time [s]')
ax.set_ylabel('Position [m]')
ax.set_title('Position Tracking', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 2: Velocity vs Time
ax = axes[0, 1]
ax.plot(time, true_states[:, 1], 'g-', linewidth=2, label='True Velocity')
ax.plot(time, estimates[:, 1], 'b-', linewidth=2, label='KF Estimate')
ax.set_xlabel('Time [s]')
ax.set_ylabel('Velocity [m/s]')
ax.set_title('Velocity Estimation (unobserved!)', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 3: Position Error
ax = axes[1, 0]
ax.plot(time, position_errors, 'r-', linewidth=2)
ax.axhline(y=measurement_std, color='k', linestyle='--', label=f'Meas. noise')
ax.set_xlabel('Time [s]')
ax.set_ylabel('Position Error [m]')
ax.set_title('Position Error', fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)

# Plot 4: Velocity Error
ax = axes[1, 1]
ax.plot(time, velocity_errors, 'r-', linewidth=2)
ax.set_xlabel('Time [s]')
ax.set_ylabel('Velocity Error [m/s]')
ax.set_title('Velocity Error', fontweight='bold')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nKey observations:")
print("  - KF provides smoother estimates than raw measurements")
print("  - Velocity is estimated even though only position is observed")
print("  - Error stays below measurement noise after convergence")


---

# Summary

In this notebook, we covered:

1. **Least Squares Methods** (Eq. 3.1-3.4)
   - Linear LS for overdetermined systems
   - Weighted LS for heterogeneous measurements
   - Robust LS for outlier rejection

2. **Kalman Filter** (Eq. 3.11-3.19)
   - Prediction and update steps
   - Estimating unobserved states (velocity from position)
   - Uncertainty quantification

**Next Steps:** 
- Explore Extended Kalman Filter (EKF) for nonlinear systems
- Try the full comparison example: `python -m ch3_estimators.example_comparison`
- Apply to Chapter 4 (RF Positioning), Chapter 6 (Dead Reckoning), and Chapter 8 (Sensor Fusion)!
